In [14]:
ls 

9379d3dc-0-dataset.tar.gz        clustering.zip
Archive.zip                      dataset/
Clustering.ipynb                 prediction-72204.csv
Complaint_Status_Tracking.ipynb  prediction.csv
README.md                        prediction2.csv
c3cc8568-0-dataset/              prediction2.txt
c3cc8568-0-dataset.tar.gz


In [15]:
path = "c3cc8568-0-dataset"

In [16]:
from glob import glob
import csv
from time import time
from datetime import datetime

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils import np_utils

import os
print(os.listdir(path))

['test.csv', 'train.csv', 'sample_submission.csv']


In [17]:
train_data = pd.read_csv('{}/train.csv'.format(path))
test_data = pd.read_csv('{}/test.csv'.format(path))

In [18]:

train_data = train_data.drop('Consumer-complaint-summary', axis = 1)
test_data = test_data.drop('Consumer-complaint-summary', axis = 1)

print("Training data: ")
train_data.head()

Training data: 


,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,Closed with non-monetary relief,No
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,Closed with explanation,No
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No


In [19]:
print("Testing data: ")
test_data.head()

Testing data: 


,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes
0,Te-1,8/18/2016,Bank account or service,"Account opening, closing, or management",Company has responded to the consumer and the ...,8/18/2016,No
1,Te-2,4/18/2016,Debt collection,Communication tactics,Company believes it acted appropriately as aut...,4/20/2016,No
2,Te-3,3/23/2016,Credit reporting,Incorrect information on credit report,NaN,3/23/2016,No
3,Te-4,6/26/2017,Student loan,Dealing with your lender or servicer,NaN,6/26/2017,NaN
4,Te-5,5/13/2016,Credit reporting,Incorrect information on credit report,Company has responded to the consumer and the ...,5/13/2016,No


In [20]:
status = train_data['Complaint-Status']
train_data = train_data.drop('Complaint-Status', axis = 1)
status.unique()

array(['Closed with explanation', 'Closed with non-monetary relief',
       'Closed', 'Closed with monetary relief', 'Untimely response'],
      dtype=object)

In [21]:
n_records = len(train_data)
print("Total number of records: {}".format(n_records))

Total number of records: 43266


In [22]:
def preprocess(data):
#     data = data.drop('Consumer-complaint-summary', axis = 1)
    columns = ['Transaction-Type', 'Complaint-reason', 'Company-response', 'Consumer-disputes']
    for column in columns:
        data[column] = data[column].replace(np.nan, '', regex=True)
        print("Columns: ", column)
        print(data[column].unique())

    data['duration'] = data.apply(lambda row: (
        datetime.strptime(row['Date-received'], '%m/%d/%Y') 
        - 
        datetime.strptime(row['Date-received'], '%m/%d/%Y')
    ).days, axis=1)

    data = data.drop('Date-received', axis = 1)
    data = data.drop('Date-sent-to-company', axis = 1)
    data = data.drop('Complaint-ID', axis = 1)
    
    columns.append('duration')
    return pd.get_dummies(data, prefix=columns, columns=columns)

In [23]:
test_id = test_data['Complaint-ID']

train_data['train'] = 1
test_data['train'] = 0

combined = train_data.append(test_data)

combined = preprocess(combined)

# combined =pd.concat([combined, preprocess(combined)], axis = 1)

train = combined[combined['train']==1]
test = combined[combined['train']==0]

train = train.drop('train', axis=1)
test = test.drop('train', axis=1)

Columns:  Transaction-Type
['Mortgage' 'Credit reporting' 'Bank account or service' 'Debt collection'
 'Credit card' 'Consumer Loan' 'Credit card or prepaid card'
 'Credit reporting, credit repair services, or other personal consumer reports'
 'Student loan' 'Payday loan' 'Vehicle loan or lease' 'Prepaid card'
 'Payday loan, title loan, or personal loan'
 'Money transfer, virtual currency, or money service'
 'Checking or savings account' 'Other financial service' 'Money transfers'
 'Virtual currency']
Columns:  Complaint-reason
['Loan servicing, payments, escrow account'
 'Incorrect information on credit report' 'Using a debit or ATM card'
 "Cont'd attempts collect debt not owed" 'Payoff process'
 'Loan modification,collection,foreclosure'
 'Problems caused by my funds being low'
 'Credit card protection / Debt protection' 'Managing the loan or lease'
 'Problem when making payments' 'Incorrect information on your report'
 'False statements or representation' 'Disclosure verification of

In [24]:
train.head()

,Transaction-Type_Bank account or service,Transaction-Type_Checking or savings account,Transaction-Type_Consumer Loan,Transaction-Type_Credit card,Transaction-Type_Credit card or prepaid card,Transaction-Type_Credit reporting,"Transaction-Type_Credit reporting, credit repair services, or other personal consumer reports",Transaction-Type_Debt collection,"Transaction-Type_Money transfer, virtual currency, or money service",Transaction-Type_Money transfers,...,Company-response_Company believes it acted appropriately as authorized by contract or law,Company-response_Company believes the complaint is the result of a misunderstanding,Company-response_Company can't verify or dispute the facts in the complaint,Company-response_Company chooses not to provide a public response,Company-response_Company disputes the facts presented in the complaint,Company-response_Company has responded to the consumer and the CFPB and chooses not to provide a public response,Consumer-disputes_,Consumer-disputes_No,Consumer-disputes_Yes,duration_0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [25]:
test.head()

,Transaction-Type_Bank account or service,Transaction-Type_Checking or savings account,Transaction-Type_Consumer Loan,Transaction-Type_Credit card,Transaction-Type_Credit card or prepaid card,Transaction-Type_Credit reporting,"Transaction-Type_Credit reporting, credit repair services, or other personal consumer reports",Transaction-Type_Debt collection,"Transaction-Type_Money transfer, virtual currency, or money service",Transaction-Type_Money transfers,...,Company-response_Company believes it acted appropriately as authorized by contract or law,Company-response_Company believes the complaint is the result of a misunderstanding,Company-response_Company can't verify or dispute the facts in the complaint,Company-response_Company chooses not to provide a public response,Company-response_Company disputes the facts presented in the complaint,Company-response_Company has responded to the consumer and the CFPB and chooses not to provide a public response,Consumer-disputes_,Consumer-disputes_No,Consumer-disputes_Yes,duration_0
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
1,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,1
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [26]:
status.unique()

array(['Closed with explanation', 'Closed with non-monetary relief',
       'Closed', 'Closed with monetary relief', 'Untimely response'],
      dtype=object)

In [27]:
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train, 
                                                    status, 
                                                    test_size = 0.2, 
                                                    random_state = 0)
print(X_train.shape)
print(y_train.shape)

(34612, 185)
(34612,)


In [28]:
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
model = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1, presort=True, min_impurity_split=10**-10))

# from sklearn.ensemble import RandomForestClassifier
# model= RandomForestClassifier(random_state=1)

model.fit(train, status)
model.score(X_test,y_test)

/Users/vkv/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/vkv/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/vkv/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use the min_impurity_decrease parameter instead.
  DeprecationWarning)
/Users/vkv/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/tree/tree.py:282: DeprecationWarning: The min_impurity_split parameter is deprecated and will be removed in version 0.21. Use

0.793737000231107

In [68]:
predictions = model.predict(test)

In [69]:
with open('prediction.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Complaint-ID", "Complaint-Status"])
    for i in range(len(predictions)):
#         print(predictions[i])
        writer.writerow([test_id[i], predictions[i]])